In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

In [60]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

mnist_data = datasets.MNIST(
    root="data/", train=True, transform=transforms.ToTensor(), download=True
)
dataloader = DataLoader(mnist_data, batch_size=64, shuffle=True)

In [61]:
#Check the dataset
total_train = 0
for features, label in dataloader:
    total_train = total_train + features.shape[0]

print("The shape of this dataset: {} ".format(features.shape))
print("# of the dataset that I downloaded = {} ".format(total_train))

The shape of this dataset: torch.Size([32, 1, 28, 28]) 
# of the dataset that I downloaded = 60000 


In [62]:
features.reshape(-1, 28*28).shape

torch.Size([32, 784])

In [63]:
class Generator(nn.Module):
    """
    A generative neural network model for generating images using the DCGAN architecture.

    Args:
        latent_space (int): The dimensionality of the latent space noise vector. Default is 100.

    Attributes:
        latent_space (int): The dimensionality of the latent space noise vector.
        model (nn.Sequential): The generator model composed of several layers.

    Example:
        >>> generator = Generator(latent_space=100)
        >>> noise = torch.randn(64, 100)
        >>> generated_images = generator(noise)
    """

    def __init__(self, latent_space=100):
        """
        Initialize the Generator.

        Args:
            latent_space (int, optional): The dimensionality of the latent space noise vector. Default is 100.
        """
        self.latent_space = latent_space
        super(Generator, self).__init__()
        layers_config = [
            (self.latent_space, 256, 0.02),
            (256, 512, 0.02),
            (512, 1024, 0.02),
            (1024, 28 * 28),
        ]
        self.model = self.generate_layer(layers_config=layers_config)

    def generate_layer(self, layers_config):
        """
        Create the layers of the generator model based on the provided configuration.

        Args:
            layers_config (list): A list of tuples specifying the layer configurations.

        Returns:
            nn.Sequential: A sequential model containing the specified layers.

        Example:
            >>> layers_config = [(100, 256, 0.02), (256, 512, 0.02), (512, 1024, 0.02), (1024, 28*28)]
            >>> generator = Generator()
            >>> generator_model = generator.generate_layer(layers_config)
        """
        layers = OrderedDict()
        for index, (input_feature, out_feature, negative_slope) in enumerate(
            layers_config[:-1]
        ):
            layers[f"layer_{index}"] = nn.Linear(
                in_features=input_feature, out_features=out_feature
            )
            layers[f"layer_{index}_activation"] = nn.LeakyReLU(
                negative_slope=negative_slope
            )

        layers[f"output_layer"] = nn.Linear(
            in_features=layers_config[-1][0], out_features=layers_config[-1][1]
        )
        layers[f"output_layer_activation"] = nn.Tanh()

        return nn.Sequential(layers)

    def forward(self, x):
        """
        Forward pass of the generator model.

        Args:
            x (torch.Tensor): Input noise tensor sampled from the latent space.

        Returns:
            torch.Tensor: Generated images.

        Example:
            >>> noise = torch.randn(64, 100)
            >>> generated_images = generator(noise)
        """
        if x is not None:
            x = self.model(x)
        else:
            x = -1

        return x.reshape(-1, 1, 28, 28)

In [64]:
generator = Generator()

print(generator.parameters)

<bound method Module.parameters of Generator(
  (model): Sequential(
    (layer_0): Linear(in_features=100, out_features=256, bias=True)
    (layer_0_activation): LeakyReLU(negative_slope=0.02)
    (layer_1): Linear(in_features=256, out_features=512, bias=True)
    (layer_1_activation): LeakyReLU(negative_slope=0.02)
    (layer_2): Linear(in_features=512, out_features=1024, bias=True)
    (layer_2_activation): LeakyReLU(negative_slope=0.02)
    (output_layer): Linear(in_features=1024, out_features=784, bias=True)
    (output_layer_activation): Tanh()
  )
)>


In [65]:
# Total number of parameters of Generator

total_parameters = 0
for layer, params in generator.named_parameters():
    total_parameters += params.numel()
    print("Layer: {} & # of parameters: {} ".format(layer, params.numel()))


print(
    "\nTotal number of parameters of generator is {} ".format(total_parameters).upper()
)

Layer: model.layer_0.weight & # of parameters: 25600 
Layer: model.layer_0.bias & # of parameters: 256 
Layer: model.layer_1.weight & # of parameters: 131072 
Layer: model.layer_1.bias & # of parameters: 512 
Layer: model.layer_2.weight & # of parameters: 524288 
Layer: model.layer_2.bias & # of parameters: 1024 
Layer: model.output_layer.weight & # of parameters: 802816 
Layer: model.output_layer.bias & # of parameters: 784 

TOTAL NUMBER OF PARAMETERS OF GENERATOR IS 1486352 


In [66]:
class Discriminator(nn.Module):
    """
    A Discriminator class representing a neural network model for distinguishing real images from generated ones.

    This class inherits from nn.Module and constructs a neural network discriminator model suitable for a Generative
    Adversarial Network (GAN). The discriminator is designed to take flattened image inputs (such as those from the
    MNIST dataset) and output a single value indicating the likelihood that the image is real.

    Attributes:
        model (torch.nn.Sequential): A sequential container of layers forming the discriminator network. The architecture
                                     is defined based on the layers configuration provided in `layers_config`.

    Methods:
        forward(x): Defines the forward pass of the discriminator.

    Parameters:
        layers_config (list of tuples): Each tuple in the list contains configuration for a layer in the model,
                                        including the number of input features, output features, and the negative
                                        slope for the LeakyReLU activation function. The last layer uses a Sigmoid
                                        activation function instead of LeakyReLU.
    """

    def __init__(self):
        super(Discriminator, self).__init__()

        layers_config = [
            (28 * 28, 512, 0.02),
            (512, 256, 0.02),
            (
                256,
                1,
            ),  # No negative slope for the last layer as it uses a Sigmoid activation
        ]
        self.model = self.discriminator_block(layers_config)

    def discriminator_block(self, layers_config):
        """
        Builds the discriminator block based on the provided layers configuration.

        Args:
            layers_config (list of tuples): Configuration for each layer in the discriminator model.

        Returns:
            torch.nn.Sequential: A sequential container of layers forming the discriminator network.
        """
        layers = OrderedDict()
        for index, (input_features, output_features, negative_slope) in enumerate(
            layers_config[:-1]
        ):
            layers[f"{index}_layer"] = nn.Linear(
                in_features=input_features, out_features=output_features
            )
            layers[f"{index}_activation"] = nn.LeakyReLU(negative_slope=negative_slope)

        # Output layer with Sigmoid activation
        layers["output_layer"] = nn.Linear(
            in_features=layers_config[-1][0], out_features=layers_config[-1][1]
        )
        layers["output_activation"] = nn.Sigmoid()

        return nn.Sequential(layers)

    def forward(self, x):
        """
        Defines the forward pass of the discriminator.

        Args:
            x (torch.Tensor): The input tensor containing the image data.

        Returns:
            torch.Tensor: The output of the discriminator, representing the probability that the input image is real.
        """
        x = x.view(-1, 28 * 28)  # Flatten the input
        return self.model(x)

In [67]:
discriminator = Discriminator()

print(discriminator.parameters)

<bound method Module.parameters of Discriminator(
  (model): Sequential(
    (0_layer): Linear(in_features=784, out_features=512, bias=True)
    (0_activation): LeakyReLU(negative_slope=0.02)
    (1_layer): Linear(in_features=512, out_features=256, bias=True)
    (1_activation): LeakyReLU(negative_slope=0.02)
    (output_layer): Linear(in_features=256, out_features=1, bias=True)
    (output_activation): Sigmoid()
  )
)>


In [68]:
# Total number of parameters of Discriminator

total_parameters = 0
for layer, params in discriminator.named_parameters():
    total_parameters+=params.numel()
    print("Layer: {} & # of parameters: {} ".format(layer, params.numel()))
    

print("\nTotal number of parameters of discriminator is {} ".format(total_parameters).upper()) 

Layer: model.0_layer.weight & # of parameters: 401408 
Layer: model.0_layer.bias & # of parameters: 512 
Layer: model.1_layer.weight & # of parameters: 131072 
Layer: model.1_layer.bias & # of parameters: 256 
Layer: model.output_layer.weight & # of parameters: 256 
Layer: model.output_layer.bias & # of parameters: 1 

TOTAL NUMBER OF PARAMETERS OF DISCRIMINATOR IS 533505 
